In [1]:
import numpy as np
import sympy as sp
from sympy.interactive.printing import init_printing
from IPython.display import display, Math

In [2]:
init_printing(use_unicode=False, wrap_line=False)

# Wedderburn 

In [22]:
C = np.random.randint(size = (6,4), high = 500, low = -500)
sp.Matrix(C)

[-38   204   -272  358 ]
[                      ]
[434   -493  341   497 ]
[                      ]
[408   139   -496  348 ]
[                      ]
[-227  -17   230   -24 ]
[                      ]
[-316  163   -246  188 ]
[                      ]
[-229  303   -370  -116]

In [537]:
def wedd(A, sv = False, xk):
    B = A
    r = np.linalg.matrix_rank(A)
    (m,n) = A.shape
    i1 = np.identity(max(m,n))[np.ix_(list(range(m)),list(range(n)))]
    i2 = i1.T
    k = 0
    ans = []
    ans.append(B)
    ind = np.nonzero(B)
    non = (ind[0][0],ind[1][0])
    s = []
    w = []
    ei = []
    ej = []
    while(abs(B[non]) > 1e-5):
        ei.append(i2[:,[non[1]]])
        ej.append(i1[:,[non[0]]])
        w.append(float(np.matmul(ej[k].T,np.matmul(B,ei[k]))))
        eij = np.matmul(ei[k],ej[k].T)
        s.append((np.matmul(B,np.matmul(eij,B))/w[k]))
        B = B - s[k]
        B[abs(B)<1e-8] = 0
        ans.append(B)
        ind = np.nonzero(B)
        try:
            non = (ind[0][0],ind[1][0])
        except IndexError:
            non = (0,0)
        k += 1
    return(ans,s)

SyntaxError: non-default argument follows default argument (<ipython-input-537-169af9222923>, line 1)

In [34]:
a = wedd(C)

### The Wedderburn Matrices

In [35]:
for i in range(len(a[0])):
    t = sp.latex(sp.Matrix(a[0][i]))
    r = r'{}'.format(t)
    display(Math(r))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [36]:
np.linalg.matrix_rank(a[0][0])

4

### Matrix A as a sum of rank one Matrices

In [37]:
for i in range(len(a[1])):
    t = sp.latex(sp.Matrix(a[1][i]))
    r = r'{}'.format(t)
    display(Math(r))
(m,n) = a[1][0].shape
p = np.zeros((m,n))
for i in range(len(a[1])):
    p = p + a[1][i]
sp.Matrix(p)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

[-261.0   51.0   -43.0   105.0 ]
[                              ]
[-292.0  -109.0  -214.0  222.0 ]
[                              ]
[-387.0  -51.0   484.0    99.0 ]
[                              ]
[-302.0   78.0   -61.0   392.0 ]
[                              ]
[ 41.0   381.0   -66.0   119.0 ]
[                              ]
[-299.0   98.0   -227.0  -354.0]

In [38]:
np.linalg.matrix_rank(a[1][3])

1

# Biconjugation

In [667]:
def prin(A):
    (m,n) = A.shape
    l = []
    flag = 0
    for i in range(min(m,n)):
        l.append(i)
        if np.linalg.det(A[np.ix_(l,l)]) == 0:
            return(0)
    return(1)

In [668]:
def bic(A, s, x1 = np.matrix([]), y1 = np.matrix([])):
    m = A.shape[0]
    n = A.shape[1]
    r = np.linalg.matrix_rank(A)
    u = []
    v = []
    w = []
    if s == 'trapldu':
        X = np.identity(n)[np.ix_(list(range(n)),list(range(r)))]
        Y = np.identity(m)[np.ix_(list(range(m)),list(range(r)))]
    elif s == 'qr':
        X = np.identity(n)
        Y = A
    elif s == 'cholesky':
        X = np.identity(m)
        Y = np.identity(m)
    elif s == 'sv':
        X = x1
        Y = y1
    else:
        print('Invalid Choice')
        return()
    u.append(X[:,[0]])
    v.append(Y[:,[0]])
    for k in range(1,r):
        sum1 = np.zeros((n,1))
        sum2 = np.zeros((m,1))
        for j in range(k):
            sum1 = sum1 + ((float(np.matmul(np.matmul(v[j].T,A),X[:,[k]]))/float(np.matmul(np.matmul(v[j].T,A),u[j])))*u[j])
            sum2 = sum2 + ((float(np.matmul(np.matmul(Y[:,[k]].T,A),u[j]))/float(np.matmul(np.matmul(v[j].T,A),u[j])))*v[j])
        u.append(X[:,[k]] - sum1)
        v.append(Y[:,[k]] - sum2)
    for k in range(r):
        w.append(float(np.matmul(np.matmul(v[k].T,A),u[k])))
    U = np.concatenate(u, axis = 1)
    V = np.concatenate(v, axis = 1)
    W = np.diag(w)
    if s == 'cholesky':
        W1 = np.sqrt(W)
        R = np.matmul(W1,np.linalg.inv(U))
        return(R)
    elif s == 'qr':
        W1 = np.sqrt(W)
        R = np.matmul(W1,np.linalg.inv(U))
        Q = np.matmul(V,np.linalg.inv(W1))
        return(Q,R)
    elif s == 'trapldu':
        L = np.matmul(A,U)
        D = np.linalg.inv(W)
        U = np.matmul(V.T,A)
        return(L,D,U)
    elif s == 'sv':
        return(U,W,V)
    else:
        print('Invalid Choice')

# QR Decomposition

In [697]:
C = np.random.randint(size = (4,3), high = 500, low = -500)
sp.Matrix(C)

[355    8     60 ]
[                ]
[155   -131  -230]
[                ]
[279   260   -390]
[                ]
[-134   76   -187]

In [698]:
R = bic(C, 'qr')

In [699]:
t = sp.latex(sp.Eq(sp.Matrix(np.matmul(R[0],R[1])), sp.MatMul(sp.Matrix(R[0]), sp.Matrix(R[1]))))
r = r'{}'.format(t)
display(Math(r))

<IPython.core.display.Math object>

In [700]:
sp.Matrix(np.matmul(R[0],R[1]))

[355.0    8.0     60.0 ]
[                      ]
[155.0   -131.0  -230.0]
[                      ]
[279.0   260.0   -390.0]
[                      ]
[-134.0   76.0   -187.0]

In [96]:
print('Q = ')
sp.Matrix(R[0])

Q = 


[-0.460182935780716   0.61223413653157   -0.556818751237619]
[                                                          ]
[0.432342686996829   0.231524491573826   -0.510526950059498]
[                                                          ]
[-0.766425672403469  -0.122217720189985  0.144655627375159 ]
[                                                          ]
[0.117911641908226   0.746075465846778   0.639053879576538 ]

In [97]:
print('R = ')
sp.Matrix(R[1])

R = 


[610.62672722376  -84.406721327665  -406.873772344652]
[                                                    ]
[      0.0        611.134604972353  149.846070140819 ]
[                                                    ]
[      0.0              0.0         501.019848550319 ]

# Cholesky Factorisation

In [701]:
A = np.random.randint(low = 1,high = 100, size = (3,3))
C = np.dot(A,A.T)

In [702]:
sp.Matrix(C)

[15530  8084  9301]
[                 ]
[8084   4280  5126]
[                 ]
[9301   5126  7118]

In [703]:
R = bic(C, 'cholesky')

In [704]:
t = sp.latex(sp.Eq(sp.Matrix(C), sp.MatMul(sp.Matrix(R.T), sp.Matrix(R))))
r = r'{}'.format(t)
display(Math(r))

<IPython.core.display.Math object>

In [102]:
sp.Matrix(np.matmul(R.T,R))

[6731.0   9613.0   10754.0]
[                         ]
[9613.0   14171.0  16318.0]
[                         ]
[10754.0  16318.0  19694.0]

In [103]:
print('R = ')
sp.Matrix(R)

R = 


[82.0426718238747  117.170733062384  131.078129964931]
[                                                    ]
[      0.0         21.0242553643056  45.6367850816537]
[                                                    ]
[      0.0               0.0         20.7318038845559]

# LDU Facatorisation

In [716]:
C = np.random.randint(size = (7,3), high = 500, low = -500)
sp.Matrix(C)

[-92   -104  -340]
[                ]
[115   -126  -132]
[                ]
[-408   17   300 ]
[                ]
[-495  -36   167 ]
[                ]
[-17   -287  -55 ]
[                ]
[-22   -342  -11 ]
[                ]
[433   176   -293]

In [717]:
R = bic(C,'trapldu')

In [718]:
a0 = R[0]
a2 = R[2]
a1 = R[1]

In [719]:
a0[abs(a0)<1e-10] = 0
a2[abs(a2)<1e-10] = 0
a1[abs(a1)<1e-10] = 0

In [720]:
t = sp.latex(sp.Eq(sp.Matrix(np.matmul(np.matmul(R[0],R[1]),R[2])), sp.MatMul(sp.MatMul(sp.Matrix(a0), sp.Matrix(a1), sp.Matrix(a2)))))
r = r'{}'.format(t)
display(Math(r))

<IPython.core.display.Math object>

In [109]:
sp.Matrix(np.matmul(np.matmul(R[0],R[1]),R[2]))

[-377.0       -464.0            -216.0      ]
[                                           ]
[-361.0       -101.0            -456.0      ]
[                                           ]
[273.0        456.0        -63.0000000000001]
[                                           ]
[167.0        -198.0             116.0      ]
[                                           ]
[-453.0  84.0000000000001       -336.0      ]
[                                           ]
[100.0        466.0              -95.0      ]
[                                           ]
[-124.0       -395.0            -471.0      ]

In [110]:
print('L = ')
sp.Matrix(a0)

L = 


[-377.0         0.0                0.0       ]
[                                            ]
[-361.0  343.307692307692          0.0       ]
[                                            ]
[273.0         120.0        -132.319755537871]
[                                            ]
[167.0   -403.538461538462  -272.563313682616]
[                                            ]
[-453.0  641.538461538462   389.161890486529 ]
[                                            ]
[100.0   342.923076923077   96.5935314887929 ]
[                                            ]
[-124.0  -242.384615384615  -575.873697142018]

In [111]:
print('D =')
sp.Matrix(a1)

D =


[-0.0026525198938992          0.0                  0.0         ]
[                                                              ]
[        0.0          0.00291283889760251          0.0         ]
[                                                              ]
[        0.0                  0.0          -0.00755745047997609]

In [112]:
print('U =')
sp.Matrix(a2)

U =


[-377.0       -464.0            -216.0      ]
[                                           ]
[ 0.0    343.307692307692  -249.167108753316]
[                                           ]
[ 0.0          0.0         -132.319755537871]

In [411]:
C = np.matrix([[1,2,3],[5,3,4],[8,3,4],[1,2,3],[5,3,4]])
sp.Matrix(C)

[1  2  3]
[       ]
[5  3  4]
[       ]
[8  3  4]
[       ]
[1  2  3]
[       ]
[5  3  4]

# SVD

In [3]:
from scipy.optimize import minimize

In [4]:
C = np.random.randint(size = (5,4), high = 500, low = -500)
sp.Matrix(C)

[-134  132   69   -357]
[                     ]
[-316  469   129  323 ]
[                     ]
[196   411   274  -108]
[                     ]
[410   -460  487  142 ]
[                     ]
[ 48   283   343  -166]

In [5]:
def maxim2(C):
    m,n = C.shape
    def objective1(x):
        x1 = np.matrix(x).T
        return(-np.linalg.norm(np.matmul(C,x1)))
    def constraint1(x):
        x1 = np.array(x)
        return(np.dot(x1,x1) - 1)

    x0 = [0 for i in list(range(n))]
    con1 = {'type': 'eq', 'fun': constraint1}

    cons = [con1]
    sol = minimize(objective1,x0,method='SLSQP',constraints=cons)
    return(sol)

In [6]:
def sv1(C):
    m,n = C.shape
    r = np.linalg.matrix_rank(C)
    u = []
    v = []
    w = []
    for i in range(r):
        u.append(np.matrix(maxim2(C).x).T)
        v.append(np.matrix(np.matmul(C,u[i])/np.linalg.norm(np.matmul(C,u[i]))))
        w.append(float(np.matmul(v[i].T,np.matmul(C,u[i]))))
        C = C - (w[i]*(np.matmul(v[i],u[i].T)))
    U = np.concatenate(u, axis = 1)
    V = np.concatenate(v, axis = 1)
    print(w)
    W = np.diag(w)
    return(U,W,V)

In [7]:
U,W,V = sv1(C)

[917.8286212393269, 757.0957332626533, 541.1374275784985, 242.18166224374866]


In [8]:
sp.Matrix(C)

[-134  132   69   -357]
[                     ]
[-316  469   129  323 ]
[                     ]
[196   411   274  -108]
[                     ]
[410   -460  487  142 ]
[                     ]
[ 48   283   343  -166]

In [9]:
ans = sp.Matrix(np.matmul(np.matmul(V,W),U.T))

In [10]:
t = sp.latex(sp.Eq(sp.Matrix(ans), sp.MatMul(sp.MatMul(sp.Matrix(V), sp.Matrix(W)), sp.Matrix(U.T))))
r = r'{}'.format(t)
display(Math(r))

<IPython.core.display.Math object>

In [11]:
np.linalg.svd(C, full_matrices = False)

(array([[-0.20568041, -0.12284701,  0.58870998, -0.60984498],
        [-0.56339027, -0.17062726, -0.71662221, -0.32850122],
        [-0.23602536, -0.61691647,  0.13765703,  0.63230437],
        [ 0.74092017, -0.50459157, -0.28799412, -0.27239011],
        [-0.18872283, -0.5662123 ,  0.19489252, -0.21487406]]),
 array([917.82862105, 757.09573326, 541.13742758, 242.18166224]),
 array([[ 0.49470031, -0.85268436,  0.15749776,  0.0582704 ],
        [-0.37590576, -0.36708517, -0.84463478,  0.10264223],
        [ 0.12163918, -0.02619886, -0.1617154 , -0.97896152],
        [ 0.77406096,  0.37079844, -0.48542216,  0.1664436 ]]))